# **BirdCLEF+ 2025**

## **Initial Setup, Exploration and Baseline**

_Insert intro_

## **Imports and Setup**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, gc 
from pathlib import Path
from tqdm.notebook import tqdm

import librosa
import librosa.display
import soundfile as sf

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

In [4]:
# Set configuration
class Config:
    SEED = 42
    SAMPLE_RATE = 32000
    DATA_PATH = Path("/kaggle/input/birdclef-2025")

cfg = Config()

In [6]:
# Function to seed everything to ensure reproducibility
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # Change to true if input sizes are kept constant

seed_everything(cfg.SEED)

In [8]:
# Device check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## **Explore Data Structure**

Let's attempt to review the various file types, including the metadata. According to the competition's data tab, the file `train.csv` contains the metadata we are looking for.

In [9]:
# Files in the base data path
print(f"Files in the base data path include: {os.listdir(cfg.DATA_PATH)}")

Files in the base data path include: ['sample_submission.csv', 'taxonomy.csv', 'train_audio', 'train_soundscapes', 'train.csv', 'recording_location.txt', 'test_soundscapes']


In [10]:
# Taking a closer look at the meta data
metadata_path = cfg.DATA_PATH / "train.csv"

if metadata_path.exists():
    train_df = pd.read_csv(metadata_path)
    print(train_df.head(15))
    print("\nMetadata Columns:", train_df.columns)
    print("\nTraining Samples:", len(train_df))
    print("\nUnique Species:", train_df['primary_label'].nunique())
    print()
    print("\nSecondary Species Labels(Recordist Marked):", train_df['secondary_labels'].nunique())

   primary_label              secondary_labels                    type  \
0        1139490                          ['']                    ['']   
1        1139490                          ['']                    ['']   
2        1192948                          ['']                    ['']   
3        1192948                          ['']                    ['']   
4        1192948                          ['']                    ['']   
5        1192948                          ['']                    ['']   
6        1194042                          ['']                    ['']   
7        1194042                          ['']                    ['']   
8        1194042                          ['']                    ['']   
9         126247  ['65448', '22976', '476538']  ['advertisement call']   
10        126247                          ['']                    ['']   
11        126247                          ['']                    ['']   
12        126247                      